In [14]:
import re
import gensim
import os
import pickle 

In [3]:
path_to_data = '../data/raw/'

In [4]:
#In the scripts we can find that the dialogs start with the character name in the middle, 
#because we are working with txt it is 25 white spaces
#Also we can find the dialogs always end with \\n\\n
dialog_start = ' ' * 25
dialog_end = '\\n\\n'

In [8]:
#We will divide scripts into scenery descriptions and dialogs.
dialogs = []
scenery = []

In [6]:
#function for tokenization and cleaning raw dialogs and sceneries 
def tokenize_clean(lines):
    cleaned_lines = []
    for line in lines:
        # first remove all placeholders we used for dividing script
        line = line.replace('<CENTER>', ' ') 
        line = line.replace('<BREAK>', ' ')
        line = line.replace('<DIALOG>', ' ')
        line = gensim.utils.simple_preprocess(line) # cleans and tokenizes text
        if len(line) != 0:
            cleaned_lines.append(line)
    return cleaned_lines

In [16]:
def extract_dialog_scenery(script):
    script = script.replace(dialog_start, '<CENTER>') #replace whitespaces with placeholder
    script = script.replace('\\n\\n', '<BREAK>') #replace \\n\\n with placeholder
    dialog_patern = '<CENTER>(.*?)<BREAK>'
    raw_dialogs = re.findall('<CENTER>(.*?)<BREAK>', script, re.DOTALL) #find all dialog lines
    script = re.sub('<CENTER>(.*?)<BREAK>', '<DIALOG>', script) #replace all dialog lines with <DIALOG>
    raw_scenary = script.split('<DIALOG>') #divide  scenaries
    clean_dialogs = tokenize_clean(raw_dialogs)
    clean_dialogs = [['<NEW_LINE>'] + line for line in clean_dialogs] #Add indication of new line
    clean_scenery = tokenize_clean(raw_scenary)
    return clean_dialogs, clean_scenery     

In [17]:
for file in os.listdir(path_to_data):
    script = str(open(path_to_data + file, 'rb').read()) # read script
    new_dialogs, new_scenery = extract_dialog_scenery(script)
    dialogs += new_dialogs
    scenery += new_scenery

In [18]:
#save clean data
pickle.dump(dialogs, open('../data/dialogs.pkl', 'wb'))
pickle.dump(scenery, open('../data/scenery.pkl', 'wb'))